In [9]:
import pandas as pd
import numpy as np
import math

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; height:100% !important; }</style>"))
  
# creating the Numpy array
array = np.array([
    [1, 1, 0], 
    [2, 4, 0], 
    [3, 9, 1], 
    [4, 6, 1], 
    [4, 3, 0],              
    [7, 1, 0],
    [4, 9, 1],
    [0, 0, 0],
    [1, 0, 0],
    [2, 2, 0],
]).astype(float)
   
# creating a list of column names
column_values = ['x', 'y', 'sum>10']
  
# creating the dataframe
df = pd.DataFrame(
    data = array, 
    columns = column_values
)
  
# displaying the dataframe
df


,x,y,sum>10
0,1.0,1.0,0.0
1,2.0,4.0,0.0
2,3.0,9.0,1.0
3,4.0,6.0,1.0
4,4.0,3.0,0.0
5,7.0,1.0,0.0
6,4.0,9.0,1.0
7,0.0,0.0,0.0
8,1.0,0.0,0.0
9,2.0,2.0,0.0


In [10]:
# setting up default clusters
clusters = []
for i in range(len(df)):
    cluster = [i]
    clusters.append(cluster)
    
clusters

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]

In [11]:
# distance matrix
for i in range(len(df)):
    df[i] = (df['x']-df.loc[df.index[i], 'x'])**2 + (df['y']-df.loc[df.index[i], 'y'])**2
    
df

,x,y,sum>10,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,0.0,0.0,10.0,68.0,34.0,13.0,36.0,73.0,2.0,1.0,2.0
1,2.0,4.0,0.0,10.0,0.0,26.0,8.0,5.0,34.0,29.0,20.0,17.0,4.0
2,3.0,9.0,1.0,68.0,26.0,0.0,10.0,37.0,80.0,1.0,90.0,85.0,50.0
3,4.0,6.0,1.0,34.0,8.0,10.0,0.0,9.0,34.0,9.0,52.0,45.0,20.0
4,4.0,3.0,0.0,13.0,5.0,37.0,9.0,0.0,13.0,36.0,25.0,18.0,5.0
5,7.0,1.0,0.0,36.0,34.0,80.0,34.0,13.0,0.0,73.0,50.0,37.0,26.0
6,4.0,9.0,1.0,73.0,29.0,1.0,9.0,36.0,73.0,0.0,97.0,90.0,53.0
7,0.0,0.0,0.0,2.0,20.0,90.0,52.0,25.0,50.0,97.0,0.0,1.0,8.0
8,1.0,0.0,0.0,1.0,17.0,85.0,45.0,18.0,37.0,90.0,1.0,0.0,5.0
9,2.0,2.0,0.0,2.0,4.0,50.0,20.0,5.0,26.0,53.0,8.0,5.0,0.0


In [12]:
merged_clusters = []

for iteration in range(len(df)-1):
    
    # finding minimum value
    min_val = 100
    min_x = 0
    min_y = 0

    print("DISTANCES")
    for i in range(len(df)):
        for j in range(len(df)):
            if (i > j):
                print('{0:.2f}'.format(df.loc[df.index[i], j]), end="\t")
                if (min_val > df.loc[df.index[i], j] and df.loc[df.index[i], j] > 0):
                    min_val = df.loc[df.index[i], j]
                    min_x = j
                    min_y = i
            else:
                print("-", end ="\t")
                df.loc[df.index[i], j] = 0
        print()
    print()

    print("minimum: ", min_val)
    print("merged cluster 1: ", clusters[min_x], " <= ", min_x)
    print("merged cluster 2: ", clusters[min_y], " <= ", min_y)
    print()
    
    merged_clusters.append(min_x)
    merged_clusters.append(min_y)

    # merging clusters
    new_cluster = []
    for c in clusters[min_x]:
        if c not in new_cluster:
            new_cluster.append(c)
    for c in clusters[min_y]:
        if c not in new_cluster:
            new_cluster.append(c)
    clusters.append(new_cluster)

    print("clusters List")
    print(clusters)


    # adding new cluster to dataframe
    new_x_sum = 0
    new_y_sum = 0
    count_0 = 0
    count_1 = 0

    for cluster in new_cluster:
        new_x_sum += df.loc[df.index[cluster], 'x']
        new_y_sum += df.loc[df.index[cluster], 'y']

        if(df.loc[df.index[cluster], 'sum>10'] == 0.0):
            count_0 += 1
        else:
            count_1 += 1

    new_target = 0
    if (count_0 > count_1):
        new_target = 0.0
    else:
        new_target = 1.0

    new_row = {'x':new_x_sum/len(new_cluster), 'y':new_y_sum/len(new_cluster), 'sum>10':new_target}
    df = df.append(new_row, ignore_index=True)

    for i in range(len(df)-1):
        df.loc[df.index[len(df)-1], i] = math.sqrt((df.loc[df.index[i], 'x']-df.loc[df.index[len(df)-1], 'x'])**2 + (df.loc[df.index[i], 'y']-df.loc[df.index[len(df)-1], 'y'])**2)

    df[len(df)-1] = 0.00000

    # removing previous clusters
    
    for cluster in merged_clusters:
        df[cluster] = 0
        for i in range(len(df)):
            df.loc[df.index[cluster], i] = 0


DISTANCES
-	-	-	-	-	-	-	-	-	-	
10.00	-	-	-	-	-	-	-	-	-	
68.00	26.00	-	-	-	-	-	-	-	-	
34.00	8.00	10.00	-	-	-	-	-	-	-	
13.00	5.00	37.00	9.00	-	-	-	-	-	-	
36.00	34.00	80.00	34.00	13.00	-	-	-	-	-	
73.00	29.00	1.00	9.00	36.00	73.00	-	-	-	-	
2.00	20.00	90.00	52.00	25.00	50.00	97.00	-	-	-	
1.00	17.00	85.00	45.00	18.00	37.00	90.00	1.00	-	-	
2.00	4.00	50.00	20.00	5.00	26.00	53.00	8.00	5.00	-	

minimum:  1.0
merged cluster 1:  [2]  <=  2
merged cluster 2:  [6]  <=  6

clusters List
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [2, 6]]
DISTANCES
-	-	-	-	-	-	-	-	-	-	-	
10.00	-	-	-	-	-	-	-	-	-	-	
0.00	0.00	-	-	-	-	-	-	-	-	-	
34.00	8.00	0.00	-	-	-	-	-	-	-	-	
13.00	5.00	0.00	9.00	-	-	-	-	-	-	-	
36.00	34.00	0.00	34.00	13.00	-	-	-	-	-	-	
0.00	0.00	0.00	0.00	0.00	0.00	-	-	-	-	-	
2.00	20.00	0.00	52.00	25.00	50.00	0.00	-	-	-	-	
1.00	17.00	0.00	45.00	18.00	37.00	0.00	1.00	-	-	-	
2.00	4.00	0.00	20.00	5.00	26.00	0.00	8.00	5.00	-	-	
8.38	5.22	0.00	3.04	6.02	8.73	0.00	9.66	9.34	7.16	-	

minimum:  1.0
merg

C:\Users\nisha\AppData\Local\Temp\ipykernel_9196\1739553547.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\nisha\AppData\Local\Temp\ipykernel_9196\1739553547.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\nisha\AppData\Local\Temp\ipykernel_9196\1739553547.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\nisha\AppData\Local\Temp\ipykernel_9196\1739553547.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\nisha\AppDa